In [41]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

pd.set_option("display.precision", 1)

# set a random seed
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)


## On récupère la donnée

In [42]:
test_customers = pd.read_csv('data/test_customers.csv')
test_locations = pd.read_csv('data/test_locations.csv')

train_customers = pd.read_csv('data/train_customers.csv')
train_locations = pd.read_csv('data/train_locations.csv')

orders = pd.read_csv('data/orders.csv')
vendors = pd.read_csv('data/vendors.csv')


/tmp/ipykernel_24992/3811830009.py:7: DtypeWarning: Columns (15,16,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  orders = pd.read_csv('data/orders.csv')


In [43]:
print('train_customers shape: ', train_customers.shape)
print('test_customers shape: ', test_customers.shape)
train_customers.head()

train_customers shape:  (34674, 8)
test_customers shape:  (9768, 8)


,akeed_customer_id,gender,dob,status,verified,language,created_at,updated_at
0,TCHWPBT,Male,NaN,1,1,EN,2018-02-07 19:16:23,2018-02-07 19:16:23
1,ZGFSYCZ,Male,NaN,1,1,EN,2018-02-09 12:04:42,2018-02-09 12:04:41
2,S2ALZFL,Male,NaN,0,1,EN,2018-03-14 18:31:43,2018-03-14 18:31:42
3,952DBJQ,Male,NaN,1,1,EN,2018-03-15 19:47:07,2018-03-15 19:47:07
4,1IX6FXS,Male,NaN,1,1,EN,2018-03-15 19:57:01,2018-03-15 19:57:01


In [44]:
print('train_locations shape: ', train_locations.shape)
print('test_locations shape: ', test_locations.shape)
train_locations.head()

train_locations shape:  (59503, 5)
test_locations shape:  (16720, 5)


,customer_id,location_number,location_type,latitude,longitude
0,02SFNJH,0,NaN,1.7,-78.8
1,02SFNJH,1,NaN,1.7,0.8
2,02SFNJH,2,NaN,-0.5,0.7
3,RU43CXC,0,Home,0.1,0.4
4,BDFBPRD,0,NaN,2.5,0.7


In [45]:
print('orders shape: ', orders.shape)
orders.head()

orders shape:  (135303, 26)


,akeed_order_id,customer_id,item_count,grand_total,payment_mode,promo_code,vendor_discount_amount,promo_code_discount_percentage,is_favorite,is_rated,...,driver_accepted_time,ready_for_pickup_time,picked_up_time,delivered_time,delivery_date,vendor_id,created_at,LOCATION_NUMBER,LOCATION_TYPE,CID X LOC_NUM X VENDOR
0,163238.0,92PEE24,1.0,7.6,2,NaN,0.0,NaN,NaN,No,...,NaN,NaN,NaN,NaN,2019-07-31 05:30:00,105,2019-08-01 05:30:16,0,NaN,92PEE24 X 0 X 105
1,163240.0,QS68UD8,1.0,8.7,1,NaN,0.0,NaN,NaN,No,...,NaN,NaN,NaN,NaN,2019-07-31 05:30:00,294,2019-08-01 05:31:10,0,Work,QS68UD8 X 0 X 294
2,163241.0,MB7VY5F,2.0,14.4,1,NaN,0.0,NaN,NaN,No,...,NaN,NaN,NaN,NaN,2019-07-31 05:30:00,83,2019-08-01 05:31:33,0,NaN,MB7VY5F X 0 X 83
3,163244.0,KDJ951Y,1.0,7.1,1,NaN,0.0,NaN,NaN,No,...,NaN,NaN,NaN,NaN,2019-07-31 05:30:00,90,2019-08-01 05:34:54,0,Home,KDJ951Y X 0 X 90
4,163245.0,BAL0RVT,4.0,27.2,1,NaN,0.0,NaN,NaN,No,...,NaN,NaN,NaN,NaN,2019-07-31 05:30:00,83,2019-08-01 05:35:51,0,Work,BAL0RVT X 0 X 83


In [46]:
print('vendors shape: ', vendors.shape)
vendors.head()

vendors shape:  (100, 59)


,id,authentication_id,latitude,longitude,vendor_category_en,vendor_category_id,delivery_charge,serving_distance,is_open,OpeningTime,...,open_close_flags,vendor_tag,vendor_tag_name,one_click_vendor,country_id,city_id,created_at,updated_at,device_type,display_orders
0,4,118597.0,-0.6,0.8,Restaurants,2.0,0.0,6.0,1.0,11:00AM-11:30PM,...,1.0,"2,4,5,8,91,22,12,24,16,23","Arabic,Breakfast,Burgers,Desserts,Free Deliver...",Y,1.0,1.0,2018-01-30 14:42:04,2020-04-07 15:12:43,3,1
1,13,118608.0,-0.5,0.7,Restaurants,2.0,0.7,5.0,1.0,08:30AM-10:30PM,...,1.0,"4,41,51,34,27,15,24,16,28","Breakfast,Cakes,Crepes,Italian,Pasta,Pizzas,Sa...",Y,1.0,1.0,2018-05-03 12:32:06,2020-04-05 20:46:03,3,1
2,20,118616.0,-0.4,0.6,Restaurants,2.0,0.0,8.0,1.0,08:00AM-10:45PM,...,1.0,"4,8,91,10","Breakfast,Desserts,Free Delivery,Indian",Y,1.0,1.0,2018-05-04 22:28:22,2020-04-07 16:35:55,3,1
3,23,118619.0,-0.6,0.8,Restaurants,2.0,0.0,5.0,1.0,10:59AM-10:30PM,...,1.0,"5,8,30,24","Burgers,Desserts,Fries,Salads",Y,1.0,1.0,2018-05-06 19:20:48,2020-04-02 00:56:17,3,1
4,28,118624.0,0.5,0.6,Restaurants,2.0,0.7,15.0,1.0,11:00AM-11:45PM,...,1.0,5,Burgers,Y,1.0,1.0,2018-05-17 22:12:38,2020-04-05 15:57:41,3,1


### On merge les customer et leur localisation

In [47]:
# merge train_customers and train_locations
train = pd.merge(train_customers, train_locations, left_on='akeed_customer_id', right_on='customer_id')
train.drop(['akeed_customer_id'], axis=1, inplace=True)
train['CID X LOC'] = train['customer_id'] + ' X ' + train['location_number'].astype(str)

print('train shape: ', train.shape)
train.head()

train shape:  (58213, 13)


,gender,dob,status,verified,language,created_at,updated_at,customer_id,location_number,location_type,latitude,longitude,CID X LOC
0,Male,NaN,1,1,EN,2018-02-07 19:16:23,2018-02-07 19:16:23,TCHWPBT,0,Work,-96.4,-67.2,TCHWPBT X 0
1,Male,NaN,1,1,EN,2018-02-07 19:16:23,2018-02-07 19:16:23,TCHWPBT,1,Home,-96.6,-67.2,TCHWPBT X 1
2,Male,NaN,1,1,EN,2018-02-07 19:16:23,2018-02-07 19:16:23,TCHWPBT,2,NaN,-0.1,-78.6,TCHWPBT X 2
3,Male,NaN,1,1,EN,2018-02-09 12:04:42,2018-02-09 12:04:41,ZGFSYCZ,0,Home,-0.2,-78.6,ZGFSYCZ X 0
4,Male,NaN,1,1,EN,2018-02-09 12:04:42,2018-02-09 12:04:41,ZGFSYCZ,1,Home,0.2,-78.6,ZGFSYCZ X 1


In [48]:
# merge test_customers and test_locations
test = pd.merge(test_customers, test_locations, left_on='akeed_customer_id', right_on='customer_id')

print('test shape: ', test.shape)
test.head()

test shape:  (16333, 13)


,akeed_customer_id,gender,dob,status,verified,language,created_at,updated_at,customer_id,location_number,location_type,latitude,longitude
0,ICE2DJP,Male,NaN,1,1,EN,2018-02-07 16:45:36,2018-02-07 16:45:36,ICE2DJP,0,NaN,-9.6e+01,-67.2
1,ICE2DJP,Male,NaN,1,1,EN,2018-02-07 16:45:36,2018-02-07 16:45:36,ICE2DJP,1,NaN,3.9e-02,-78.6
2,ICE2DJP,Male,NaN,1,1,EN,2018-02-07 16:45:36,2018-02-07 16:45:36,ICE2DJP,2,Other,-9.5e+01,43.7
3,ICE2DJP,Male,NaN,1,1,EN,2018-02-07 16:45:36,2018-02-07 16:45:36,ICE2DJP,3,Other,-9.6e+01,43.6
4,ICE2DJP,Male,NaN,1,1,EN,2018-02-07 16:45:36,2018-02-07 16:45:36,ICE2DJP,4,Work,2.0e+02,44.1


### On arrange la donnée pour notre réseau

on créé une matrice, avec en ligne les clients et en colonne les vendeurs, on met a 1 les couples (client, vendeur) apparaissants dans orders

In [49]:
# we create a matrix with all the possible combinations of CID as lines and VENDOR as columns
# we fill the matrix with 0 if the combination is not in check_orders
# we fill the matrix with 1 if the combination is in check_orders
"""
# we create a list of all the CID_X_LOC
CID_X_LOC_list = train['CID X LOC'].unique()
print('CID_X_LOC_list shape: ', CID_X_LOC_list.shape)

# we create a list of all the VENDOR
VENDOR_list = vendors['id'].unique()
print('VENDOR_list shape: ', VENDOR_list.shape)

# we create a matrix with all the possible combinations of CID as lines and VENDOR as columns
matrix = np.zeros((len(CID_X_LOC_list), len(VENDOR_list)))

# we create a list of all the CID X LOC X VENDOR in orders
CID_X_LOC_X_VENDOR = orders['CID X LOC_NUM X VENDOR'].unique().astype(str)
print('CID_X_LOC_X_VENDOR shape: ', CID_X_LOC_X_VENDOR.shape)

# we fill the matrix with 1 if the combination is in orders
for e in CID_X_LOC_X_VENDOR:
    CID_X_LOC = e.split(' X ')[0] + ' X ' + e.split(' X ')[1]
    VENDOR = int(e.split(' X ')[2])
    matrix[CID_X_LOC_list == CID_X_LOC, VENDOR_list == VENDOR] = 1

# check by counting the number of 1 in the matrix
print('number of 1 in the matrix: ', np.sum(matrix))

print('matrix shape: ', matrix.shape)
"""


"\n# we create a list of all the CID_X_LOC\nCID_X_LOC_list = train['CID X LOC'].unique()\nprint('CID_X_LOC_list shape: ', CID_X_LOC_list.shape)\n\n# we create a list of all the VENDOR\nVENDOR_list = vendors['id'].unique()\nprint('VENDOR_list shape: ', VENDOR_list.shape)\n\n# we create a matrix with all the possible combinations of CID as lines and VENDOR as columns\nmatrix = np.zeros((len(CID_X_LOC_list), len(VENDOR_list)))\n\n# we create a list of all the CID X LOC X VENDOR in orders\nCID_X_LOC_X_VENDOR = orders['CID X LOC_NUM X VENDOR'].unique().astype(str)\nprint('CID_X_LOC_X_VENDOR shape: ', CID_X_LOC_X_VENDOR.shape)\n\n# we fill the matrix with 1 if the combination is in orders\nfor e in CID_X_LOC_X_VENDOR:\n    CID_X_LOC = e.split(' X ')[0] + ' X ' + e.split(' X ')[1]\n    VENDOR = int(e.split(' X ')[2])\n    matrix[CID_X_LOC_list == CID_X_LOC, VENDOR_list == VENDOR] = 1\n\n# check by counting the number of 1 in the matrix\nprint('number of 1 in the matrix: ', np.sum(matrix))\n\n

In [50]:
# we create a dataframe with the matrix
#matrix_df = pd.DataFrame(matrix, index=CID_X_LOC_list, columns=VENDOR_list)
#matrix_df.head(10)

### Nettoyage de la donnée

Nettoyage des customer X location

In [51]:
# label encoding
from sklearn.preprocessing import LabelEncoder

# print train columns
print('train columns: ', train.columns)
print('train shape: ', train.shape)
print('number of missing values per column:')
display(train.isnull().sum())
print('df_input types: ', train.dtypes)

def clean_df(df, le):
    # no warning
    pd.options.mode.chained_assignment = None

    # df cleaning
    new_df = df[['latitude', 'longitude', 'location_type', 'status', 'gender', 'language']]

    # we fill the missing values with the mean
    new_df['latitude'].fillna(new_df['latitude'].mean(), inplace=True)
    new_df['longitude'].fillna(new_df['longitude'].mean(), inplace=True)
    new_df['language'].fillna('EN', inplace=True)
    new_df['gender'].fillna('None', inplace=True)
    new_df['location_type'].fillna('None', inplace=True)

    # fit and transform the location_type column
    new_df['location_type'] = le.fit_transform(new_df['location_type'])
    new_df['language'] = le.fit_transform(new_df['language'])
    new_df['gender'] = le.fit_transform(new_df['gender'])

    # set back the warning
    pd.options.mode.chained_assignment = 'warn'

    return new_df


# create a LabelEncoder object
le = LabelEncoder()

train = clean_df(train, le)
test = clean_df(test, le)
train.head()

train columns:  Index(['gender', 'dob', 'status', 'verified', 'language', 'created_at',
       'updated_at', 'customer_id', 'location_number', 'location_type',
       'latitude', 'longitude', 'CID X LOC'],
      dtype='object')
train shape:  (58213, 13)
number of missing values per column:


gender             17051
dob                53018
status                 0
verified               0
language           15769
created_at             0
updated_at             0
customer_id            0
location_number        0
location_type      26593
latitude               6
longitude              6
CID X LOC              0
dtype: int64

df_input types:  gender              object
dob                float64
status               int64
verified             int64
language            object
created_at          object
updated_at          object
customer_id         object
location_number      int64
location_type       object
latitude           float64
longitude          float64
CID X LOC           object
dtype: object


,latitude,longitude,location_type,status,gender,language
0,-96.4,-67.2,3,1,7,0
1,-96.6,-67.2,0,1,7,0
2,-0.1,-78.6,1,1,7,0
3,-0.2,-78.6,0,1,7,0
4,0.2,-78.6,0,1,7,0


Nettoyage des vendors

In [52]:
# print vendors columns
print('vendors columns: ', vendors.columns)
print('vendors shape: ', vendors.shape)
print('number of missing values per column:')
display(vendors.isnull().sum())
print('df_input types: ', vendors.dtypes)

vendors = vendors[['latitude', 'longitude', 'vendor_category_id', 'delivery_charge', 'serving_distance', 'is_open', 'prepration_time', 'discount_percentage', 'rank', 'language', 'vendor_rating', 'device_type']]

vendors['language'].fillna('EN', inplace=True)

vendors['language'] = le.fit_transform(vendors['language'])

vendors.head()

vendors columns:  Index(['id', 'authentication_id', 'latitude', 'longitude',
       'vendor_category_en', 'vendor_category_id', 'delivery_charge',
       'serving_distance', 'is_open', 'OpeningTime', 'OpeningTime2',
       'prepration_time', 'commission', 'is_akeed_delivering',
       'discount_percentage', 'status', 'verified', 'rank', 'language',
       'vendor_rating', 'sunday_from_time1', 'sunday_to_time1',
       'sunday_from_time2', 'sunday_to_time2', 'monday_from_time1',
       'monday_to_time1', 'monday_from_time2', 'monday_to_time2',
       'tuesday_from_time1', 'tuesday_to_time1', 'tuesday_from_time2',
       'tuesday_to_time2', 'wednesday_from_time1', 'wednesday_to_time1',
       'wednesday_from_time2', 'wednesday_to_time2', 'thursday_from_time1',
       'thursday_to_time1', 'thursday_from_time2', 'thursday_to_time2',
       'friday_from_time1', 'friday_to_time1', 'friday_from_time2',
       'friday_to_time2', 'saturday_from_time1', 'saturday_to_time1',
       'saturday_from

id                       0
authentication_id        0
latitude                 0
longitude                0
vendor_category_en       0
vendor_category_id       0
delivery_charge          0
serving_distance         0
is_open                  0
OpeningTime              9
OpeningTime2             9
prepration_time          0
commission              15
is_akeed_delivering      0
discount_percentage      0
status                   0
verified                 0
rank                     0
language                15
vendor_rating            0
sunday_from_time1        1
sunday_to_time1          1
sunday_from_time2       58
sunday_to_time2         58
monday_from_time1        0
monday_to_time1          0
monday_from_time2       58
monday_to_time2         58
tuesday_from_time1       1
tuesday_to_time1         1
tuesday_from_time2      59
tuesday_to_time2        59
wednesday_from_time1     0
wednesday_to_time1       0
wednesday_from_time2    58
wednesday_to_time2      58
thursday_from_time1      1
t

df_input types:  id                        int64
authentication_id       float64
latitude                float64
longitude               float64
vendor_category_en       object
vendor_category_id      float64
delivery_charge         float64
serving_distance        float64
is_open                 float64
OpeningTime              object
OpeningTime2             object
prepration_time           int64
commission              float64
is_akeed_delivering      object
discount_percentage     float64
status                  float64
verified                  int64
rank                      int64
language                 object
vendor_rating           float64
sunday_from_time1        object
sunday_to_time1          object
sunday_from_time2        object
sunday_to_time2          object
monday_from_time1        object
monday_to_time1          object
monday_from_time2        object
monday_to_time2          object
tuesday_from_time1       object
tuesday_to_time1         object
tuesday_from_time2     

,latitude,longitude,vendor_category_id,delivery_charge,serving_distance,is_open,prepration_time,discount_percentage,rank,language,vendor_rating,device_type
0,-0.6,0.8,2.0,0.0,6.0,1.0,15,0.0,11,0,4.4,3
1,-0.5,0.7,2.0,0.7,5.0,1.0,14,0.0,11,0,4.7,3
2,-0.4,0.6,2.0,0.0,8.0,1.0,19,0.0,1,0,4.5,3
3,-0.6,0.8,2.0,0.0,5.0,1.0,16,0.0,11,0,4.5,3
4,0.5,0.6,2.0,0.7,15.0,1.0,10,0.0,11,0,4.4,3


### Fusion des deux vecteurs

In [53]:
# merge train and vendors
# for each client, we create 

KeyError: 'id'

### Création des data splits

In [ ]:
# data split
train, test = train_test_split(df_input, test_size=0.2, random_state=42, shuffle=True)
print('test shape: ', test.shape)
print('train shape: ', train.shape)

y_train = all_pairs['PREDICTION']
y_train, y_test = train_test_split(y_train, test_size=0.2, random_state=42, shuffle=True)

NameError: name 'df_input' is not defined

### Creation du modèle

In [ ]:
# creation of an ai model
# it takes as input a dataframe with the columns as in df_input
# it returns a dataframe with the columns as in all_pairs and a column PREDICTION with the prediction

from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization

model = keras.Sequential()
model.add(Input(shape=(df_input.shape[1],)))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

model.summary()


Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_62 (Dense)            (None, 256)               3072      
                                                                 
 dense_63 (Dense)            (None, 128)               32896     
                                                                 
 dense_64 (Dense)            (None, 64)                8256      
                                                                 
 dense_65 (Dense)            (None, 1)                 65        
                                                                 
Total params: 44,289
Trainable params: 44,289
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# fit the model
model.fit(train, y_train, epochs=10, batch_size=32, validation_data=(test, y_test))


Epoch 1/10
4008/4008 [==============================] - 4s 792us/step - loss: 0.2456 - accuracy: 0.5572 - val_loss: 0.2421 - val_accuracy: 0.5724
Epoch 2/10
4008/4008 [==============================] - 3s 773us/step - loss: 0.2403 - accuracy: 0.5755 - val_loss: 0.2393 - val_accuracy: 0.5743
Epoch 3/10
4008/4008 [==============================] - 3s 770us/step - loss: 0.2387 - accuracy: 0.5875 - val_loss: 0.2370 - val_accuracy: 0.5917
Epoch 4/10
4008/4008 [==============================] - 3s 781us/step - loss: 0.2361 - accuracy: 0.5935 - val_loss: 0.2373 - val_accuracy: 0.5932
Epoch 5/10
4008/4008 [==============================] - 3s 782us/step - loss: 0.2352 - accuracy: 0.5974 - val_loss: 0.2342 - val_accuracy: 0.6032
Epoch 6/10
4008/4008 [==============================] - 3s 782us/step - loss: 0.2347 - accuracy: 0.6003 - val_loss: 0.2333 - val_accuracy: 0.6068
Epoch 7/10
4008/4008 [==============================] - 3s 779us/step - loss: 0.2334 - accuracy: 0.6036 - val_loss: 0.2328 -